In [2]:
import pandas as pd

# Load datasets
odds_df = pd.read_csv("cleaned_odds_data.csv")
games_df = pd.read_csv("games.csv")
teams_df = pd.read_csv("teams.csv")

# Convert date columns
odds_df['date'] = pd.to_datetime(odds_df['date'])
games_df['GAME_DATE_EST'] = pd.to_datetime(games_df['GAME_DATE_EST'])

# Map TEAM_ID to team names
team_id_to_name = teams_df.set_index("TEAM_ID")["NICKNAME"].to_dict()

# Add team names to games data
games_df["home_team_name"] = games_df["TEAM_ID_home"].map(team_id_to_name)
games_df["away_team_name"] = games_df["TEAM_ID_away"].map(team_id_to_name)

# Merge games_df with odds_df as "Team A vs Team B"
# First, convert all names to lowercase for consistent merging
odds_df["teamA"] = odds_df["teamA"].str.lower()
odds_df["teamB"] = odds_df["teamB"].str.lower()
games_df["home_team_name"] = games_df["home_team_name"].str.lower()
games_df["away_team_name"] = games_df["away_team_name"].str.lower()

# Merge using date + teamA/home and teamB/away logic
merged_df = odds_df.merge(
    games_df,
    left_on=["date", "teamA", "teamB"],
    right_on=["GAME_DATE_EST", "home_team_name", "away_team_name"],
    how="inner"
)

# Optionally: drop duplicated or irrelevant columns
merged_df = merged_df.drop(columns=["GAME_DATE_EST", "home_team_name", "away_team_name"])

# Preview result
print("Merged dataset shape:", merged_df.shape)
print("Columns:", merged_df.columns.tolist())


Merged dataset shape: (0, 32)
Columns: ['date', 'season', 'teamA', 'teamB', 'teamA_home', 'teamA_score', 'teamB_score', 'teamA_moneyLine', 'teamB_moneyLine', 'total', 'spread', 'secondHalfTotal', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS']


In [3]:
# First lowercase team names for consistency
odds_df['teamA'] = odds_df['teamA'].str.lower()
odds_df['teamB'] = odds_df['teamB'].str.lower()
games_df['home_team_name'] = games_df['TEAM_ID_home'].map(team_id_to_name).str.lower()
games_df['away_team_name'] = games_df['TEAM_ID_away'].map(team_id_to_name).str.lower()

# Merge where teamA == home team
merge1 = odds_df.merge(
    games_df,
    left_on=['date', 'teamA', 'teamB'],
    right_on=['GAME_DATE_EST', 'home_team_name', 'away_team_name'],
    how='inner'
)

# Merge where teamA == away team (i.e., flip)
merge2 = odds_df.merge(
    games_df,
    left_on=['date', 'teamA', 'teamB'],
    right_on=['GAME_DATE_EST', 'away_team_name', 'home_team_name'],
    how='inner'
)

# Combine both matches
merged_df = pd.concat([merge1, merge2], ignore_index=True)

print("Merged dataset shape:", merged_df.shape)
print("Sample rows:")
print(merged_df[['date', 'teamA', 'teamB', 'teamA_score', 'teamB_score', 'PTS_home', 'PTS_away']].head())


Merged dataset shape: (0, 35)
Sample rows:
Empty DataFrame
Columns: [date, teamA, teamB, teamA_score, teamB_score, PTS_home, PTS_away]
Index: []


In [4]:
print("Team names from odds_df:")
print(sorted(set(odds_df['teamA'].str.lower().unique()).union(set(odds_df['teamB'].str.lower().unique()))))

Team names from odds_df:
['atlanta', 'boston', 'brooklyn', 'charlotte', 'chicago', 'cleveland', 'dallas', 'denver', 'detroit', 'golden state', 'houston', 'indiana', 'la clippers', 'la lakers', 'memphis', 'miami', 'milwaukee', 'minnesota', 'new jersey', 'new orleans', 'new york', 'oklahoma city', 'orlando', 'philadelphia', 'phoenix', 'portland', 'sacramento', 'san antonio', 'seattle', 'toronto', 'utah', 'washington']


In [5]:
print("Nicknames from teams_df (mapped to games_df):")
print(sorted(set(games_df['home_team_name'].dropna().unique()).union(set(games_df['away_team_name'].dropna().unique()))))


Nicknames from teams_df (mapped to games_df):
['76ers', 'bucks', 'bulls', 'cavaliers', 'celtics', 'clippers', 'grizzlies', 'hawks', 'heat', 'hornets', 'jazz', 'kings', 'knicks', 'lakers', 'magic', 'mavericks', 'nets', 'nuggets', 'pacers', 'pelicans', 'pistons', 'raptors', 'rockets', 'spurs', 'suns', 'thunder', 'timberwolves', 'trail blazers', 'warriors', 'wizards']


In [9]:
# Step 1: Build mapping from CITY → NICKNAME
city_to_nickname = teams_df.set_index("CITY")["NICKNAME"].str.lower().to_dict()

# Step 2: Apply mapping to odds_df
odds_df['teamA_nick'] = odds_df['teamA'].str.title().map(city_to_nickname)
odds_df['teamB_nick'] = odds_df['teamB'].str.title().map(city_to_nickname)

# Step 3: Lowercase everything for merge
odds_df['teamA_nick'] = odds_df['teamA_nick'].str.lower()
odds_df['teamB_nick'] = odds_df['teamB_nick'].str.lower()
games_df['home_team_name'] = games_df['TEAM_ID_home'].map(teams_df.set_index("TEAM_ID")["NICKNAME"]).str.lower()
games_df['away_team_name'] = games_df['TEAM_ID_away'].map(teams_df.set_index("TEAM_ID")["NICKNAME"]).str.lower()

# Step 4: Try both directions of the matchup
merge1 = odds_df.merge(
    games_df,
    left_on=['date', 'teamA_nick', 'teamB_nick'],
    right_on=['GAME_DATE_EST', 'home_team_name', 'away_team_name'],
    how='inner'
)

merge2 = odds_df.merge(
    games_df,
    left_on=['date', 'teamA_nick', 'teamB_nick'],
    right_on=['GAME_DATE_EST', 'away_team_name', 'home_team_name'],
    how='inner'
)

# Step 5: Combine both
merged_df = pd.concat([merge1, merge2], ignore_index=True)

# Done
print("Merged dataset shape:", merged_df.shape)
print(merged_df[['date', 'teamA', 'teamB', 'teamA_nick', 'teamB_nick', 'PTS_home', 'PTS_away']].head())


Merged dataset shape: (15554, 37)
        date         teamA        teamB teamA_nick teamB_nick  PTS_home  \
0 2007-10-30  golden state         utah   warriors       jazz      96.0   
1 2007-10-31     cleveland       dallas  cavaliers  mavericks      74.0   
2 2007-10-31       indiana   washington     pacers    wizards     119.0   
3 2007-10-31       memphis  san antonio  grizzlies      spurs     101.0   
4 2007-10-31   new orleans   sacramento   pelicans      kings     104.0   

   PTS_away  
0     117.0  
1      92.0  
2     110.0  
3     104.0  
4      90.0  


In [10]:
print(merged_df.columns.tolist())

['date', 'season', 'teamA', 'teamB', 'teamA_home', 'teamA_score', 'teamB_score', 'teamA_moneyLine', 'teamB_moneyLine', 'total', 'spread', 'secondHalfTotal', 'teamA_nick', 'teamB_nick', 'GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'home_team_name', 'away_team_name']


In [11]:
for col in merged_df.columns:
    print(col)

date
season
teamA
teamB
teamA_home
teamA_score
teamB_score
teamA_moneyLine
teamB_moneyLine
total
spread
secondHalfTotal
teamA_nick
teamB_nick
GAME_DATE_EST
GAME_ID
GAME_STATUS_TEXT
HOME_TEAM_ID
VISITOR_TEAM_ID
SEASON
TEAM_ID_home
PTS_home
FG_PCT_home
FT_PCT_home
FG3_PCT_home
AST_home
REB_home
TEAM_ID_away
PTS_away
FG_PCT_away
FT_PCT_away
FG3_PCT_away
AST_away
REB_away
HOME_TEAM_WINS
home_team_name
away_team_name
